# Preprocessing p.2
Now that we have some knowledge about our dataset we can get get rid of some of it and then proceed to converting our data info something more TF friendly :)

### Imports 

In [ ]:
import os 
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm

lets delete 10 of each extreme cases 

In [ ]:
df = pd.read_csv('df.csv', index_col=0)

In [ ]:
df.head()

In [ ]:
dogs = df[df['label'] == 'dog']
cats = df[df['label'] == 'cat']

To maintain equal amout of dogs and cats we will delete 5 from each

In [ ]:
# This will delete only record from df that we will use to create input data 
# All images will be still on you disk for further practices
def get_indexes(df):
    indexes = df.sort_values(by='height').head().index
    indexes = np.append(indexes, df.sort_values(by='width').head().index) 
    indexes = np.append(indexes, df.sort_values(by='aspect ratio').tail().index)
    indexes = np.append(indexes, df.sort_values(by='aspect ratio').head().index)
    indexes = np.append(indexes, df.sort_values(by='range').head().index)
    indexes = np.append(indexes, df.sort_values(by='std').head().index)
    indexes = np.append(indexes, df.sort_values(by='std').tail().index)
    return np.unique(indexes)

In [ ]:
df.drop(get_indexes(dogs), inplace=True)
df.drop(get_indexes(cats), inplace=True)

In [ ]:
df.info()

We can see that we've lost about 60 images, but we got rid of some worst(probably) cases :)

### Creating training and testing data 

In [ ]:
def label_to_array(label):
    if label == 'cat':
        return [1, 0]
    elif label == 'dog':
        return [0, 1]

In [ ]:
def process_image(img):
    return img.convert('L').resize([IMG_SIZE, IMG_SIZE], Image.ANTIALIAS)

In [ ]:
def prepare_data(df):
    training_data = []
    for data in tqdm(df[['path', 'label']].head().itertuples()):
        label = label_to_array(data[2])
        path = data[1]
        img = Image.open(path, 'r')
        img = process_image(img)
        training_data.append([np.array(img), np.array(label)])
    np.random.shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data